# 見原の研究のオリジナルver
---

見原の研究のオリジナルverや

コメントを追加したverや

---
最終編集日:2023年6月2日

## ライブラリのインポート

In [ ]:
#モジュールの読み込み
from __future__ import print_function

import pandas as pd # pandasの読み込み
from pandas import Series,DataFrame # なんか知らんけどDataframe変なimportしてる（フツーpd.DataFrame)

from sklearn import svm # なんか知らんけどscikit-learn(sklearn)からsvm（サポートベクターマシン）import、ここは無くてもおk
from sklearn.model_selection import train_test_split # scikit-learn(sklearn)からtrain_test_splitをimport
from sklearn.metrics import accuracy_score # scikit-learn(sklearn)のmetrics関数からaccuracy_scoreをimport
from sklearn.preprocessing import StandardScaler # scikit-learn(sklearn)のpreprocessing関数からStandardScalerをimport

import numpy as np # numpy（なんぱい）をimporrt
import matplotlib.pyplot as plt #matplotlibのpyplot関数をimportしてpltとして定義

import tensorflow as tf # kerasのtensorflowをtfとしてimport
import seaborn as sns # seaborn（しーぼーん）をsnsとしてimport

import time # timeをimport

## データの読み込みと分割

In [ ]:
#CSVファイルの読み込み
data_set = pd.read_csv("general_data.csv",sep=",",header=0)  #pandasのread_csv関数で読み込み

#説明変数
x = DataFrame(data_set.drop(["Assignee"],axis=1)) #pandasのDataFrame関数で"Assignee"以外のカラム（列）をxに代入

#目的変数
y = DataFrame(data_set["Assignee"]) #pandasのDataFrame関数で"Assignee"のカラム（列）をxに代入

#説明変数・目的変数をそれぞれ訓練データ・テストデータに分割（ランダム），シャッフルしない場合は引数（shuffle=False）
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.5, shuffle=False)

## データの標準化（または正規化をしよう）
---

ニューラルネットワークはデータの絶対値が1以下やないと、勾配が爆発して、損失関数lossがNanになる。ほとんどの場合で必須。仮に必須やなくても、こちらの処理をしたほうが実質罰則なし正則化されるので、デメリットなく精度が上がる。やり得。


---


ただし、決定木（RandomForestやXGBoost、LightGBMなど）は、その仕組み上、標準化はなくても良い

---

In [ ]:
# データを標準化
stdsc = StandardScaler()
x_train = stdsc.fit_transform(x_train)
x_test = stdsc.transform(x_test)

# 以下すべて変な処理している（フツーしない）

#データの整形
x_train = x_train.astype(np.float)
x_test = x_test.astype(np.float)

#カテゴリデータの場合
#y_train = keras.utils.to_categorical(y_train,10)
#y_test = keras.utils.to_categorical(y_test,10)
#数値データの場合
y_train = np.array(y_train, dtype = np.float32)
y_test = np.array(y_test, dtype = np.float32)

## モデルの構築

### モデルの定義

In [ ]:
#ディープニューラルネットワークの実装1 (activation='relu')
model = tf.keras.models.Sequential() # モデルの定義

t1 = time.time() 

# 結合層(13ユニット->50ユニット)：入力次元を省略すると自動的に前の層の出力次元数を引き継ぐ
# reluやsigmoidがある
"""

以下のモデルは、入力から、100, 100, 100, 1のニューラルネット

"""
model.add(tf.keras.layers.Dense(100, activation='relu', input_shape=(13,)))
model.add(tf.keras.layers.Dropout(0.2)) # ドロップアウトは最悪0でもおk。今回のように大きい（多次元）複雑なデータにはあったほうが良き

model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2)) #ドロップアウトやバッチ正規化は正則化項なので、複雑なデータを単純化する。最適な値はデータによりけりなので、要調整。

model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

#Dropoutを緩めていくと良い（調整の際のコツ）

#目的変数がカテゴリデータの場合（10種類の場合）
#model.add(Dense(10, activation='softmax'))
#目的変数が数値データの場合（出力層．次元1，つまり一つの値を出力する．）
model.add(tf.keras.layers.Dense(1))

model.summary()
print("\n")

### モデルの学習

In [ ]:
#ここで、どのように学習をするかを定義


#ディープニューラルネットワークの実装2
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
#勾配法には、'adam'や'sgd'という方法もある
#metrics=['accuracy']
#metrics=['mae']

In [ ]:
#ここで、誤差逆伝播法で学習。
#以下はミニバッチを100、エポックを100、学習経過の表示を1エポックごと、教師データを(x_test, y_test)と定義

#ディープニューラルネットワークの学習
#history = model.fit(x_train, y_train, batch_size=100, epochs=1000, verbose=1, validation_data=(x_test, y_test))
history = model.fit(x_train, y_train, batch_size=100, epochs=100, verbose=1, validation_data=(x_test, y_test))

In [ ]:
#ここは作成したモデルを使って予測をしている


#ディープニューラルネットワークの推論
score = model.evaluate(x_test, y_test, verbose=1)
print("\n")
print("Test Loss:",score[0])
print("Test MSE:",score[1])


## 可視化（図式化）

In [ ]:
# 精度の履歴をプロット
# 分類問題のAccuracyは，(True Positive+True Negative)/(True Positive+True Negative+False Positive+False Negative)のようなものを表す．
# 回帰の場合は誤差を最小化するのでmseを使うだけで良い．観測値と予測値の差異を数値化したものを表す．
plt.plot(history.history['mse'], label="Error for Training", lw=0.5)
plt.plot(history.history['val_mse'], label="Error for Validation", lw=0.5)
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend(loc='lower center', bbox_to_anchor=(.5, 1.0), ncol=3)
plt.savefig("error.eps")
plt.close()

# 損失の履歴をプロット
# 重みパラメータが最適な値に近づくための指標
# lossは，学習用データを与えた際の損失値のこと
# この値が小さければ小さいほど賢くなったことを表し，
# 逆に値が大きければ学習が不十分だということを表す．
# val_lossは，検証データを与えた際の損失値．意味は上記のlossと同様．
#plt.plot(history.history['loss'], label="Loss for Training", lw=0.5)
#plt.plot(history.history['val_loss'], label="Loss for Validation", lw=0.5)
#plt.xlabel('Epoch')
#plt.ylabel('Loss')
#plt.legend(loc='lower center', bbox_to_anchor=(.5, 1.0), ncol=3)
#plt.savefig("loss.eps")
#plt.close()

y_pred = model.predict(x_test)

# テストデータに対する推定結果のプロット
plt.plot(y_test, lw=0.5)
plt.plot(y_pred, lw=0.5)
plt.xlabel('Number of Faults')
plt.ylabel('Frequency of Assignee')
plt.legend(['Testing Data', 'Estimate'], loc='lower center', bbox_to_anchor=(.5, 1.0), ncol=3)
plt.savefig("frequency.eps")
plt.close()

# テストデータに対する推定結果の散布図
plt.scatter(y_test, y_pred, c='r', s=10)
plt.xlabel('Testing Data')
plt.ylabel('Estimate')
plt.legend(['Estimate (Frequency of Assignee)'], loc='lower center', bbox_to_anchor=(.5, 1.0), ncol=3)
plt.plot([0,10],[0,10])
plt.savefig("general_scatter.eps")
plt.close()

# テストデータに対する推定された累積時間データのプロット
plt.plot(np.cumsum(y_test), lw=0.5)
plt.plot(np.cumsum(y_pred), lw=0.5)
plt.xlabel('Number of Faults')
plt.ylabel('Cumulative Frequency of Assignee')
plt.legend(['Testing Data', 'Estimate'], loc='lower center', bbox_to_anchor=(.5, 1.0), ncol=3)
plt.savefig("cfrequency.eps")
plt.close()

# テストデータに対する累積時間データの散布図
plt.scatter(np.cumsum(y_test), np.cumsum(y_pred), c='r', s=10)
plt.xlabel('Testing Data')
plt.ylabel('Estimate')
plt.legend(['Estimate (Cumulative Frequency of Assignee)'], loc='lower center', bbox_to_anchor=(.5, 1.0), ncol=3)
plt.plot([0,1000],[0,1000])
plt.savefig("cscatter.eps")
plt.close()

t2 = time.time()
 
# 経過時間を表示
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")
